### TO DO
1. data preprocessing
- notes we got: For example, the vote count and popularity score variables may not be good features to use as they are more likely to be response variables. A good rule of thumb is: if there is a new movie that has not been released yet, what features will we have and which will be not have?
2. train linear regression
- use budget & time of year to keep it simple
- flag 
3. train polynomial
- same deal as above
4. train RF
- add categorical vars
5. train adaboost
- same deal

### Data Cleaning
1. do we keep 2020 in there since that year is messed up
2. how do we standardize/ can we keep reviews in there??
3. are we doing profit?

In [ ]:
SARA_FILEPATH = '/Users/sarakent/Desktop/ML 1/TMDB_movie_dataset_v11.csv'   #too large to add to git, adding file path ehre

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import numpy as np

In [ ]:
df = pd.read_csv(SARA_FILEPATH)
df.head(2)

In [ ]:
df.columns

In [ ]:
# keeping columns we are interested in
# col_lst = ['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date', 'revenue',
#           'runtime', 'adult', 'budget', 'popularity', 'genres', 'production_companies',
#           'spoken_languages']
# df = df[col_lst]
df['release_date'] = pd.to_datetime(df.release_date)
df = df[(df['status'] == 'Released') & 
        (df['budget'] > 10000) & 
        (df['release_date'].notna()) & 
        (df['genres'].notna()) &
        (df['runtime'] != 0) & 
        (df['release_date'] > '1950-01-01') &
        (df['tagline'].notna()) &
        (df['overview'].notna())]

In [ ]:
df

In [ ]:
df['profit'] = df.revenue - df.budget

In [ ]:
df.profit.mean(), df.profit.max(), df.profit.min()

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(df.release_date, df.profit)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(df.release_date, df.revenue)
plt.show()

In [ ]:
df=df[df.tagline.notna() & df.overview.notna()]

In [ ]:


# Download NLTK stopwords
# nltk.download('stopwords')
nltk.download('wordnet')
# Get the English stopwords list
stop_words = set(stopwords.words('english'))


# Function to remove stopwords and numbers from text
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Tokenize text
    words = text.split()
    
    # Remove stopwords and numbers
    filtered_words = filtered_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and not word.isdigit()]
    
    # Join the filtered words back into a single string
    processed_text = ' '.join(filtered_words)
    
    return processed_text

In [ ]:
df.reset_index(inplace=True)

In [ ]:

# Load your movie dataset
# For illustration, let's assume 'data' is your DataFrame containing movie data

# Combine text fields into one
df['text'] = df['original_title'] + ' ' + df['overview'] + ' ' + df['tagline']
ids = df['id']

# Initialize CountVectorizer to convert text into word frequency vectors
df['text'] = df['text'].apply(preprocess_text)
vectorizer = CountVectorizer()

# Fit and transform the combined text to get the word frequency vectors
word_freq_matrix = vectorizer.fit_transform(df['text'])

# Convert the word frequency matrix to a DataFrame
word_freq_df = pd.DataFrame(word_freq_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Calculate the sum of occurrences of each word across rows
word_freq_sum = word_freq_df.sum()
# Calculate the percentile threshold (e.g., top 25th percentile)
percentile_threshold = np.percentile(word_freq_sum, 99.9)  # Keep words within the top 25th percentile

# Filter to include only columns where the frequency is within the percentile threshold
significant_word_freq = word_freq_df.loc[:, word_freq_sum >= percentile_threshold]
word_freqs = pd.concat([ids, significant_word_freq], axis=1)

In [ ]:
word_freqs.head()

In [ ]:
features = df[['id', 'genres', 'title', 'budget', 'release_date', 'production_companies']]

In [ ]:
features

In [ ]:
final_df = word_freqs.join(features.set_index('id'), on='id', lsuffix='_word_freqs', rsuffix='_df')

In [ ]:
final_df.to_csv('movie.csv')